In [1]:
!!pip install mcap

['Collecting mcap',
 '  Downloading mcap-1.1.1-py3-none-any.whl (27 kB)',
 'Collecting lz4',
 '  Downloading lz4-4.3.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.3 MB)',
 '     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 6.4 MB/s eta 0:00:00',
 'Collecting zstandard',
 '  Downloading zstandard-0.22.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (5.4 MB)',
 '     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 8.4 MB/s eta 0:00:00',
 'Installing collected packages: zstandard, lz4, mcap',
 'Successfully installed lz4-4.3.3 mcap-1.1.1 zstandard-0.22.0']

In [9]:
import json
import sys
from time import time_ns

from mcap.writer import Writer,CompressionType

with open("basic.mcap", "wb") as stream:
    writer = Writer(stream,compression=CompressionType.NONE)

    writer.start()

    schema_id = writer.register_schema(
        name="sample",
        encoding="jsonschema",
        data=json.dumps(
            {
                "type": "object",
                "properties": {
                    "value": {
                        "type": "integer",
                    }
                },
            }
        ).encode(),
    )

    channel_id = writer.register_channel(
        schema_id=schema_id,
        topic="integer_topic",
        message_encoding="json",
    )

    for i in range(10):
        writer.add_message(
            channel_id=channel_id,
            log_time=time_ns(),
            data=json.dumps({"value": i}).encode("utf-8"),
            publish_time=time_ns(),
            sequence=i
        )

    writer.finish()